## Coding Assignment 4

Team:
- Olivia Dalglish (od4)
- Arindam Saha (saha2)

Contribution: 

Olivia: Part 1

Arindam: Part 2

In addition to the above, we discussed our approaches and checked each other's work.

### Part I: Gaussian Mixtures

In [1]:
import numpy as np

In [2]:
import numpy as np

THRESHOLD = 1e-6

def Estep(data, G, prob, means, Sigma):
    """
    E-step: compute the responsibility matrix

    Parameters:
        data (ndarray)
        G (int)
        prob (ndarray): mixing weights for each Gaussian component
        means (ndarray): means of Gaussian components
        Sigma (ndarray): cov matrix
    """
    # compute Gaussian PDF values for all data points and components
    n, p = data.shape
    
    Sigma_inv = np.linalg.inv(Sigma)
    det_Sigma = np.linalg.det(Sigma)
    norm_factor = 1.0 / np.sqrt((2 * np.pi) ** p * det_Sigma)
    
    diff = data[:, np.newaxis, :] - means.T
    
    exponent = -0.5 * np.einsum('nik,kl,nil->ni', diff, Sigma_inv, diff)    
    pdf_matrix = norm_factor * np.exp(exponent)
    
    # multiply by mixing weights
    weighted_pdfs = pdf_matrix * prob
    
    # normalize to get responsibilities
    resp = weighted_pdfs / weighted_pdfs.sum(axis=1, keepdims=True)
    
    return resp

def Mstep(data, resp):
    """
    M-step: update parameters based on responsibilities

    Parameters:
        data (ndarray)
        resp (ndarray): responsibility matrix from E-step, shape (n, G)
    """
    n, p = data.shape
    G = resp.shape[1]

    Nk = resp.sum(axis=0)  # sum of responsibilities for each component
    prob = Nk / n

    means = np.dot(data.T, resp) / Nk

    Sigma = np.zeros((p, p))
    for k in range(G):
        diff = data - means[:, k].T 
        Sigma += np.dot((resp[:, k][:, np.newaxis] * diff).T, diff)
    Sigma /= n

    return prob, means, Sigma

def loglik(data, G, prob, means, Sigma):
    """
    Compute the log-likelihood of the data given the current parameters of the Gaussian mixture model.
    Parameters:
        data (ndarray)
        G (int)
        prob (ndarray): mixing weights for each Gaussian component
        means (ndarray): means of Gaussian components
        Sigma (ndarray): cov matrix   
    """
    n, p = data.shape
    '''
    Sigma_inv = np.linalg.inv(Sigma)
    Sigma_det = np.linalg.det(Sigma)
    normalization_const = 1 / ((2 * np.pi) ** (p / 2) * np.sqrt(Sigma_det))
    
    # data minus means for each component
    diff = data[:, np.newaxis, :] - means.T
    
    exponent = -0.5 * np.einsum('nkp, pq, nkq -> nk', diff, Sigma_inv, diff)
    
    # gaussian densities: (n, G)
    component_densities = normalization_const * np.exp(exponent)
    '''
    Sigma_inv = np.linalg.inv(Sigma)
    det_Sigma = np.linalg.det(Sigma)
    norm_factor = 1.0 / np.sqrt((2 * np.pi) ** p * det_Sigma)
    
    diff = data[:, np.newaxis, :] - means.T
    
    exponent = -0.5 * np.einsum('nik,kl,nil->ni', diff, Sigma_inv, diff)    
    pdf_matrix = norm_factor * np.exp(exponent)
    weighted_densities = pdf_matrix * prob 
    total_density = np.sum(weighted_densities, axis=1)
    log_likelihood = np.sum(np.log(total_density))
    
    return log_likelihood

def myEM(data, G, prob, means, Sigma, itmax=100):
    """
    Runner
    
    Parameters:
        data (ndarray): data, shape (n, p)
        G (int): number of Gaussian components
        prob (ndarray): initial probability vector
        means (ndarray): initial means
        Sigma (ndarray): initial cov matrix
        itmax (int): maximum number of iterations

    Returns:
        prob (ndarray): final probability vector
        means (ndarray): final means for each Gaussian component
        Sigma (ndarray): final shared cov matrix
        loglik (float): final log-likelihood of the model
    """

    log_likelihoods = []

    for iteration in range(itmax):
        resp = Estep(data, G, prob, means, Sigma)

        prob, means, Sigma = Mstep(data, resp)

        current_loglik = loglik(data, G, prob, means, Sigma)
        log_likelihoods.append(current_loglik)

        if iteration > 0 and abs(log_likelihoods[-1] - log_likelihoods[-2]) < THRESHOLD:
            break

    return prob, means, Sigma, log_likelihoods[-1]


##### Testing

In [3]:
data = np.loadtxt("faithful.dat", skiprows=1)[:, 1:]

n, p = data.shape

##### G = 2

In [4]:
G = 2

# initialize mixing weights
p1 = 10 / n
p2 = 1 - p1
prob = np.array([p1, p2])

mu1 = np.mean(data[:10], axis=0)
mu2 = np.mean(data[10:], axis=0)
means = np.column_stack((mu1, mu2))

# initialize covariance matrix
Sigma = np.zeros((p, p))
for i in range(10):
    diff = data[i] - mu1
    Sigma += np.outer(diff, diff)
for i in range(10, n):
    diff = data[i] - mu2
    Sigma += np.outer(diff, diff)
Sigma /= n

itmax = 20

final_prob, final_means, final_Sigma, log_likelihood = myEM(data, G, prob, means, Sigma, itmax)

print("Final probabilities")
print(final_prob)

print("\nFinal means")
print(final_means)

print("Final cov matrix")
print(final_Sigma)

print("log likelihood")
print(log_likelihood)

Final probabilities
[0.04297883 0.95702117]

Final means
[[ 3.49564188  3.48743016]
 [76.79789154 70.63205853]]
Final cov matrix
[[  1.29793612  13.92433626]
 [ 13.92433626 182.58009247]]
log likelihood
-1289.5693549424104


##### G = 3

In [5]:
G = 3

# initialize mixing weights
p1 = 10 / n
p2 = 20 / n
p3 = 1 - p1 - p2
prob = np.array([p1, p2, p3])

# initialize means
mu1 = np.mean(data[:10], axis=0)
mu2 = np.mean(data[10:30], axis=0)
mu3 = np.mean(data[30:], axis=0)
means = np.column_stack((mu1, mu2, mu3))

# initialize covariance matrix
Sigma = np.zeros((p, p))
for i in range(10):
    diff = data[i] - mu1
    Sigma += np.outer(diff, diff)
for i in range(10, 30):
    diff = data[i] - mu2
    Sigma += np.outer(diff, diff)
for i in range(30, n):
    diff = data[i] - mu3
    Sigma += np.outer(diff, diff)
Sigma /= n  # Shape (p, p)

itmax = 20

final_prob, final_means, final_Sigma, log_likelihood = myEM(data, G, prob, means, Sigma, itmax)

print("Final probabilities")
print(final_prob)

print("Final means")
print(final_means)

print("Final cov matrix")
print(final_Sigma)

print("log likelihood")
print(log_likelihood)

Final probabilities
[0.04363422 0.07718656 0.87917922]
Final means
[[ 3.51006918  2.81616674  3.54564083]
 [77.10563811 63.35752634 71.25084801]]
Final cov matrix
[[  1.26015772  13.51153756]
 [ 13.51153756 177.96419105]]
log likelihood
-1289.350958862739


We can see above that our output matches the expected values.

### Part II: HMM

##### Baum-Welch Algorithm

In [6]:
def BW_onestep(data, mx, mz, w, A, B):
    # Switching data to be 0-indexed
    data = data - 1
    n = len(data)

    alpha = np.zeros((n, mz))
    alpha[0] = w * B[:, data[0]]
    for t in range(1, n):
        alpha[t] = (alpha[t - 1] @ A) * B[:, data[t]]
    
    beta = np.zeros((n, mz))
    beta[n - 1] = np.ones(mz)
    for t in range(n - 2, -1, -1):
        beta[t] = A @ (B[:, data[t + 1]] * beta[t + 1])

    gamma = np.zeros((n - 1, mz, mz))
    for t in range(n - 1):
        gamma[t] = alpha[t][:, np.newaxis] * (A * (B[:, data[t + 1]] * beta[t + 1]))

    gamma_plus = np.sum(gamma, axis=0)
    A_next = gamma_plus / np.sum(gamma_plus, axis=1)[:, np.newaxis]
    
    gamma_ti = np.vstack((np.sum(gamma, axis=2), alpha[n - 1]))
    B_next = np.zeros(B.shape)
    for l in range(mx):
        t_idxs = np.where(data == l)[0]
        B_next[:, l] = np.sum(gamma_ti[t_idxs], axis=0) / np.sum(gamma_ti, axis=0)

    return A_next, B_next


def myBW(data, mx, mz, w, A, B, niter):
    for _ in range(niter):
        A, B = BW_onestep(data, mx, mz, w, A, B)
    return A, B

##### Viterbi Algorithm

In [7]:
def myViterbi(data, mx, mz, w, A, B):
    # Switching data to be 0-indexed
    data = data - 1

    # Evaluating probabilities in logarithmic scale to correctly compare really low probabilty values

    w = np.log(w)
    A = np.log(A)
    B = np.log(B)

    n = len(data)
    delta = np.zeros((n, mz))
    delta[0] = w + B[:, data[0]]
    for t in range(1, n):
        for i in range(mz):
            delta[t, i] = np.max(delta[t - 1] + A[:, i]) + B[i, data[t]]

    Z = np.zeros(n, dtype=int)
    Z[n - 1] = np.argmax(delta[n - 1])
    for t in range(n - 2, -1, -1):
        Z[t] = np.argmax(delta[t] + A[:, Z[t + 1]])

    # Switching Z to be 1-indexed
    return Z + 1

##### Testing

##### 1.

In [8]:
data = np.loadtxt('Coding4_part2_data.txt').astype(int)

mx = 3
mz = 2

w = np.array([0.5, 0.5])
A_init = np.array([
    [0.5, 0.5],
    [0.5, 0.5],
])
B_init = np.array([
    [1/9, 3/9, 5/9],
    [1/6, 2/6, 3/6],
])

A, B = myBW(data, mx, mz, w, A_init, B_init, 100)

print(f'A: the 2-by-2 transition matrix:\n{A}\n')
print(f'B: the 2-by-3 emission matrix:\n{B}\n')


Z = myViterbi(data, 3, 2, w, A, B)
print(f'Z: {Z}')

with open('Coding4_part2_Z.txt') as fh:
    expected_Z = np.fromstring(fh.read().strip(), dtype=int, sep= ' ')

print(f'matches: {np.all(Z == expected_Z)}')


A: the 2-by-2 transition matrix:
[[0.49793938 0.50206062]
 [0.44883431 0.55116569]]

B: the 2-by-3 emission matrix:
[[0.22159897 0.20266127 0.57573976]
 [0.34175148 0.17866665 0.47958186]]

Z: [1 1 1 1 1 1 1 2 1 1 1 1 1 2 2 1 1 1 1 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1 2 1 1
 1 1 1 1 1 1 2 2 1 1 1 1 1 1 2 2 2 1 1 1 1 2 2 2 2 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1
 1 1 1 2 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 1 1 1 2 2 2 2 2 2 1 1 1 1 1 2 2 2 2 2 2 2 2 2 1 1 1 2 2 2 1 1 1 1 1
 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1]
matches: True


We can see above that our output matches the expected values.

##### 2.

In [9]:
B_init_same = np.array([
    [1/3, 1/3, 1/3],
    [1/3, 1/3, 1/3],
])
A_20, B_20 = myBW(data, mx, mz, w, A_init, B_init_same, 20)
A_100, B_100 = myBW(data, mx, mz, w, A_init, B_init_same, 100)

print(f'A_20: the 2-by-2 transition matrix:\n{A_20}\n')
print(f'B_20: the 2-by-3 emission matrix:\n{B_20}\n')

print(f'A_100: the 2-by-2 transition matrix:\n{A_100}\n')
print(f'B_100: the 2-by-3 emission matrix:\n{B_100}\n')

A_20: the 2-by-2 transition matrix:
[[0.5 0.5]
 [0.5 0.5]]

B_20: the 2-by-3 emission matrix:
[[0.285 0.19  0.525]
 [0.285 0.19  0.525]]

A_100: the 2-by-2 transition matrix:
[[0.5 0.5]
 [0.5 0.5]]

B_100: the 2-by-3 emission matrix:
[[0.285 0.19  0.525]
 [0.285 0.19  0.525]]



In [10]:
np.mean(data == 1), np.mean(data == 2), np.mean(data == 3)

(0.285, 0.19, 0.525)

We can see above that with indistinguishable latent states, Baum-Welch just converges to the probabilities of each value in the data in the emission matrix. This makes sense because it is equivalent to just having one state which emits according to the probabilities found in the data.